# Cuaderno 14: Planificación de la producción (Lot Sizing)

$\newcommand{\card}[1]{\left| #1 \right|}$
$\newcommand{\ZZ}{\mathbb{Z}_+}$
$\newcommand{\tabulatedset}[1]{\left\{ #1 \right\}}$

En los problemas de planificación de la producción (*lot sizing*) se busca determinar los niveles óptimos de producción de uno o varios bienes dentro de un ciclo productivo, considerando restricciones como la demanda, la capacidad, los costos de producción, los costos de almacenamiento, entre otras.

Consideraremos, a manera de ejemplo, la planificación de los niveles óptimos de producción de una empresa dedicada a la producción de alimento balanceado, sujeta a las siguientes condiciones:

* El plan de producción se organiza sobre un horizonte de tiempo de un año, dividido en un conjunto $T= \tabulatedset{1, \ldots, 12}$ de períodos mensuales. 
* Si en un período $t \in T$ se toma la decisión de producir, se incurre en un
  costo fijo igual a $c_t$ (en USD). 
* Para cada período $t \in T$, se conocen además la demanda estimada de balanceado en el mercado $d_t$ (en toneladas) y el costo unitario de su producción $p_t$ (en USD / tonelada). 
* En cada período se debe satisfacer toda la demanda estimada. Si en un período el nivel de producción supera a la demanda, es posible almacenar el producto excedente a un costo unitario igual a $h_t$ (USD/tonelada). 
* La producción se realiza por lotes de una tonelada de alimento balanceado. No es posible producir fracciones de un lote.
  
El problema consiste en determinar los niveles de producción mensuales que permitan satisfacer toda la demanda al menor costo posible. Para formular este problema como un programa lineal entero, definimos las siguientes variables de decisión:

* variables enteras $x_t, \, t \in T,$ que indiquen la cantidad de lotes a producir en el período $t$;
* variables binarias $y_t, \, t \in T,$ tales que $y_t = 1$ si y solamente si se toma la decisión producir en el período $t$;
* variables enteras $z_t, \, t \in T,$ que indiquen la cantidad de lotes de producto a almacenar del período $t$ al período $t+1$.

Con estas definiciones, el problema puede ser formulado como el siguiente programa lineal entero:

\begin{align*}
\min &\sum_{t \in T} (p_t x_t + c_t y_t + h_t z_t)\\ 
& \mbox{s.r.}\\
& x_1 = d_1 + z_1,\\
& x_t + z_{t-1} = d_t + z_t, \quad \forall t \in T \setminus \{1\}, \\
& x_t \leq M y_t, \quad \forall t \in T, \\
& x_t, z_t \in \ZZ, y_t \in \{0, 1\}, \quad \forall t \in T.
\end{align*}


Donde $M$ es una constante suficientemente grande como para que $x_t \leq M$ se cumpla en cualquier solución factible. Podemos fijar, por ejemplo, $M:= \sum_{t \in T} d_t$.

Vamos a implementar este programa utilizando la interfaz Python de Gurobi.

Definimos primero el conjunto $T$ y los parámetros $p$, $c$, $h$, $d$ y $M$:

In [1]:
from gurobipy import *

# Conjuntos y parametros del modelo
# El diccionario con los datos tiene el formato {t : (c_t, p_t, h_t, d_t)}
# Costos fijos, costos de produccion, costos de almacenamiento y demandas:
T, c, p, h, d = multidict({
    1 : (400, 40, 10, 20),   2 : (450, 47, 12, 20),  3 : (400, 38, 10, 25),
    4 : (470, 45,  9, 25),   5 : (450, 47, 12, 25),  6 : (400, 42, 10, 20),
    7 : (350, 33, 10, 20),   8 : (400, 39, 12, 18),  9 : (450, 45, 10, 18),
   10 : (450, 47, 12, 20),  11 : (500, 50, 15, 25), 12 : (500, 55, 15, 27)})

    
# Constante igual a la suma de todas las demandas
M = quicksum(d)

# Conjunto T \ {1}
T2 = [t for t in T if not t==1]

print("T= {}".format(T))
print("T2= {}".format(T2))
print("c= {}".format(c))
print("p= {}".format(p))
print("h= {}".format(h))
print("d= {}".format(d))
print("M= {}".format(M))

T= [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
T2= [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
c= {1: 400, 2: 450, 3: 400, 4: 470, 5: 450, 6: 400, 7: 350, 8: 400, 9: 450, 10: 450, 11: 500, 12: 500}
p= {1: 40, 2: 47, 3: 38, 4: 45, 5: 47, 6: 42, 7: 33, 8: 39, 9: 45, 10: 47, 11: 50, 12: 55}
h= {1: 10, 2: 12, 3: 10, 4: 9, 5: 12, 6: 10, 7: 10, 8: 12, 9: 10, 10: 12, 11: 15, 12: 15}
d= {1: 20, 2: 20, 3: 25, 4: 25, 5: 25, 6: 20, 7: 20, 8: 18, 9: 18, 10: 20, 11: 25, 12: 27}
M= <gurobi.LinExpr: 263.0>


Definimos ahora el objeto modelo y las variables del modelo:

In [2]:
m = Model('lot-sizing')

# lotes a producir
x = m.addVars(T, vtype = GRB.INTEGER, name="x")

# se produce / no se produce en cada periodo
y = m.addVars(T, vtype = GRB.BINARY, name="y")

# lotes a almacenar
z = m.addVars(T, vtype = GRB.INTEGER, name="z")


--------------------------------------------
--------------------------------------------

Using license file /Users/ltorres/gurobi.lic
Academic license - for non-commercial use only


Construimos la función objetivo a partir de sus tres términos:

In [3]:
# costos variables de produccion
c_produccion = x.prod(p, '*')

# costos fijos de produccion
c_fijo = y.prod(c, '*')

# costos de almacenamiento
almacenamiento = z.prod(h, '*')

m.setObjective(c_produccion + c_fijo + almacenamiento, GRB.MINIMIZE)

Finalmente, implementamos las restricciones del modelo:
1. Balance entre produccion, almacenamiento y demanda.

In [4]:
# Balance primer periodo
m.addConstr(x[1] - z[1] == d[1], "balance[1]")

# Balance periodos siguientes
m.addConstrs((x[t] + z[t-1] - z[t] == d[t] for t in T2), "balance")

{2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>,
 11: <gurobi.Constr *Awaiting Model Update*>,
 12: <gurobi.Constr *Awaiting Model Update*>}

2. Solamente puede producirse cuando $y_t =1$.

In [5]:
# y[t]=0 => x[t]=0
m.addConstrs((x[t] <= M*y[t] for t in T), "sinc_xy")

{1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>,
 11: <gurobi.Constr *Awaiting Model Update*>,
 12: <gurobi.Constr *Awaiting Model Update*>}

Optimizar el modelo:

In [6]:
m.optimize()

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 24 rows, 36 columns and 59 nonzeros
Model fingerprint: 0x46236843
Variable types: 0 continuous, 36 integer (12 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [9e+00, 5e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+01, 3e+01]
Found heuristic solution: objective 16897.000000
Presolve removed 3 rows and 4 columns
Presolve time: 0.07s
Presolved: 21 rows, 32 columns, 52 nonzeros
Variable types: 0 continuous, 32 integer (11 binary)

Root relaxation: objective 1.312651e+04, 29 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 13126.5101    0   10 16897.0000 13126.5101  22.3%     -    0s
H    0     0                    16837.000000 13126.5101  22.0%     -    0s
H    0     0                    16562.000000 13126.5101  20.7%     

Mostrar la solución: producción, demanda y almacenamiento en cada período.

In [7]:
# Extraer valores de las soluciones
vx = m.getAttr('x', x)
vy = m.getAttr('x', y)
vz = m.getAttr('x', z)

# Mostrar tabla con los valores
print('t\t\tx_t\t\ty_t\t\tz_t\t\td_t')
for t in T:
    print('{}\t\t{}\t\t{}\t\t{}\t\t{}'.format(t, vx[t], vy[t], vz[t], d[t]))

t		x_t		y_t		z_t		d_t
1		40.0		1.0		20.0		20
2		0.0		0.0		-0.0		20
3		75.0		1.0		50.0		25
4		0.0		0.0		25.0		25
5		0.0		0.0		0.0		25
6		20.0		1.0		0.0		20
7		76.0		1.0		56.0		20
8		0.0		0.0		38.0		18
9		0.0		0.0		20.0		18
10		0.0		0.0		0.0		20
11		52.0		1.0		27.0		25
12		0.0		0.0		0.0		27


## Código completo

Se reproduce a continuación el código completo del modelo anterior.

In [17]:
# Curso de implementación de programas lineales enteros
# Ejemplo: Modelo de planificación de la producción (lot sizing)
# EPN (2020)

from gurobipy import *
try:
    # Conjuntos y parametros del modelo
    # Costos fijos, costos de produccion, costos de almacenamiento y demandas:
    T, c, p, h, d = multidict({
        1 : (400, 40, 10, 20),   2 : (450, 47, 12, 20),  3 : (400, 38, 10, 25),
        4 : (470, 45,  9, 25),   5 : (450, 47, 12, 25),  6 : (400, 42, 10, 20),
        7 : (350, 33, 10, 20),   8 : (400, 39, 12, 18),  9 : (450, 45, 10, 18),
       10 : (450, 47, 12, 20),  11 : (500, 50, 15, 25), 12 : (500, 55, 15, 27)})

    
    # Constante igual a la suma de todas las demandas
    M = quicksum(d)

    # Conjunto T \ {1}
    T2 = [t for t in T if not t==1]    
    
    # Crear el objeto modelo
    m = Model('lot-sizing')

    # Variables de decision
    # lotes a producir
    x = m.addVars(T, vtype = GRB.INTEGER, name="x")

    # se produce / no se produce en cada periodo
    y = m.addVars(T, vtype = GRB.BINARY, name="y")

    # lotes a almacenar
    z = m.addVars(T, vtype = GRB.INTEGER, name="z")    

    # Funcion objetivo
    # costos variables de produccion
    c_produccion = x.prod(p, '*')

    # costos fijos de produccion
    c_fijo = y.prod(c, '*')

    # costos de almacenamiento
    almacenamiento = z.prod(h, '*')

    m.setObjective(c_produccion + c_fijo + almacenamiento, GRB.MINIMIZE)

    # Restricciones
    # Balance primer periodo
    m.addConstr(x[1] - z[1] == d[1], "balance[1]")

    # Balance periodos siguientes
    m.addConstrs((x[t] + z[t-1] - z[t] == d[t] for t in T2), "balance")
    
    # y[t]=0 => x[t]=0
    m.addConstrs((x[t] <= M*y[t] for t in T), "sinc_xy")    
    
    # Resolver el modelo
    m.optimize()
    
    # Mostrar solucion
    # Extraer valores de las soluciones
    vx = m.getAttr('x', x)
    vy = m.getAttr('x', y)
    vz = m.getAttr('x', z)

    # Mostrar tabla con los valores
    print('t\t\tx_t\t\ty_t\t\tz_t\t\td_t')
    for t in T:
        print('{}\t\t{}\t\t{}\t\t{}\t\t{}'.format(t, vx[t], vy[t], vz[t], d[t]))        
except GurobiError as e:
    print('Se produjo un error de Gurobi: codigo: ' + str(e.errno) + ": " + str(e))

except AttributeError:
    print('Se produjo un error de atributo')

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 24 rows, 36 columns and 59 nonzeros
Model fingerprint: 0x46236843
Variable types: 0 continuous, 36 integer (12 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [9e+00, 5e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+01, 3e+01]
Found heuristic solution: objective 16897.000000
Presolve removed 3 rows and 4 columns
Presolve time: 0.00s
Presolved: 21 rows, 32 columns, 52 nonzeros
Variable types: 0 continuous, 32 integer (11 binary)

Root relaxation: objective 1.312651e+04, 29 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 13126.5101    0   10 16897.0000 13126.5101  22.3%     -    0s
H    0     0                    16837.000000 13126.5101  22.0%     -    0s
H    0     0                    16562.000000 13126.5101  20.7%     